<a href="https://colab.research.google.com/github/SonOf1998/ProblemSet5/blob/main/ProblemSet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from urllib.request import urlretrieve
import requests
import zipfile
import shutil
import random

working_dir = os.getcwd()
local_zip = os.path.join(working_dir, 'main.zip')

url = 'https://github.com/SonOf1998/ProblemSet5/archive/main.zip'
urlretrieve(url,local_zip) # Load content of repository into a local zip file

# Extract zip file
zip_ref = zipfile.ZipFile(local_zip,'r') 
zip_ref.extractall()
zip_ref.close()

# Discard zip file
if os.path.exists(local_zip):
  os.remove(local_zip)

In [2]:
from tensorflow.keras.datasets import cifar10

def dataset():
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()

  # min-max normalization
  x_train = x_train / 255
  x_test  = x_test / 255

  return x_train, y_train, x_test, y_test

In [3]:
!pip install hyperas
!pip install hyperopt

In [4]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [5]:
def create_model(x_train, x_test, y_train, y_test):

  activation = {{choice(['relu', 'leakyrelu'])}}
  opti = {{choice(['rmsprop', 'adam', 'sgd'])}}
  kernel_size = {{choice([2, 3])}}
  dropout_1 = {{uniform(0, 0.5)}}
  dropout_2 = {{uniform(0, 0.5)}}
  dropout_3 = {{uniform(0, 0.5)}}
  unit_size = {{choice([64, 128, 256])}}
  batch_size = {{choice([16, 32, 64, 128, 256])}}

  early_stop = EarlyStopping(monitor='val_accuracy', patience=4)

  model = keras.Sequential()
  model.add(Conv2D(16, kernel_size, activation=activation, input_shape=(32, 32, 3)))  
  model.add(Conv2D(16, kernel_size, activation=activation))
  model.add(MaxPooling2D(2))
  model.add(Dropout(dropout_1))
  model.add(Conv2D(32, kernel_size, activation=activation))
  model.add(Conv2D(64, kernel_size, activation=activation))
  model.add(MaxPooling2D(2))
  model.add(Dropout(dropout_2))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(unit_size, activation=activation))
  model.add(Dropout(dropout_3))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=opti, loss='categorical_crossentropy', metrics=['accuracy'])
  results = model.fit(x_train, y_train, batch_size=batch_size, epochs=100, verbose=0,
                      validation_data=(x_test, y_test), callbacks=[early_stop], shuffle=True)

  best_val_acc = np.amax(result.history['val_accuracy']) 

  with open('hyperas-log.csv', 'a') as csv_file:
      csv_file.write(str(kernel_size) + ';')
      csv_file.write(str(dropout_1) + ';')
      csv_file.write(str(dropout_2) + ';')
      csv_file.write(str(dropout_3) + ';')
      csv_file.write(str(activation) + ';')
      csv_file.write(str(unit_size) + ';')
      csv_file.write(str(opti) + ';')
      csv_file.write(str(batch_size) + ';')
      csv_file.write(str(best_val_acc) + '\n')


  return {'loss': -best_val_acc, 'status': STATUS_OK, 'model': model}

In [6]:
with open('hyperas-log.csv', 'w') as csv_file:
  csv_file.write("kernel_size" + ';')
  csv_file.write("dropout_1" + ';')
  csv_file.write("dropout_2" + ';')
  csv_file.write("dropout_3" + ';')
  csv_file.write("activation" + ';')
  csv_file.write("unit_size" + ';')
  csv_file.write("opti" + ';')
  csv_file.write("batch_size" + ';')
  csv_file.write("best_val_acc" + '\n')

In [8]:
best_run, best_model = optim.minimize(model=create_model, 
                                      data=dataset, algo=tpe.suggest, 
                                      max_evals=100, trials=Trials(), 
                                      notebook_name="ProblemSet5", 
                                      verbose=True)

FileNotFoundError: ignored

In [ ]:
import pandas
df = pandas.read_csv('hyperas-log.csv', delimiter=';')
df.sort_values(by=['best_val_acc'], ascending=False)